In [1]:
%load_ext autoreload
%autoreload 2
import os
import glob
import json

outs = {}


for filename in sorted([f for f in glob.glob("../output/*.json") if "test" not in f]): 
    model_name = os.path.basename(filename).split(".")[0]
    print(model_name)
    with open(filename) as f:
        outs[model_name] = json.load(f)

bertin
beto-cased-10000
beto-cased-20000
beto-cased-2500
beto-cased-5000
beto-cased
beto-uncased-10000
beto-uncased-20000
beto-uncased-2500
beto-uncased-5000
beto-uncased
checkpoint-100k
checkpoint-124k
checkpoint-46k
checkpoint-62k
checkpoint-87k
roberta-bne
robertuito-cased-574k
robertuito-cased
robertuito-uncased-200k


In [2]:
import pandas as pd
import numpy as np

resume = []
mean_resume = []
task_metrics = {
    "context_hate": ["eval_mean_f1", "eval_hate_f1"],
    "hate": ["eval_hateful_f1", "eval_macro_f1"],
    "sentiment": ["eval_macro_f1", "eval_micro_f1"],
    "emotion": ["eval_macro_f1", "eval_micro_f1"],
    "irony": ["eval_ironic_f1", "eval_macro_f1"],
}

    
for model_name, output in outs.items():
    line = {
        "model": model_name, 
    }

    mean_line = {
        "model": model_name,
    }

    for task, metrics in task_metrics.items():
        try:
            for metric in metrics:
                arr = np.array([evaluation[metric] for evaluation in output[task]])
                metric_name = metric.replace("eval_", "")
                mean_line[task+" "+metric_name] = arr.mean()
                line[task+" "+metric_name] = f"{arr.mean():.4f} +- {arr.std():.3f}"
        except KeyError as e:
            print(e, "not in ", model_name)
            continue
    resume.append(line)
    mean_resume.append(mean_line)

order = [
    "bertin",
    "beto-uncased",
    "beto-uncased-2500",
    "beto-uncased-5000",
    "beto-uncased-10000",
    "beto-uncased-20000",
    #"checkpoint-46k",
    #"checkpoint-62k",
    #"checkpoint-87k",
    #"checkpoint-100k",
    #"checkpoint-124k",
    "robertuito-uncased-200k",
    "beto-cased",
    "beto-cased-2500",
    "beto-cased-5000",
    "beto-cased-10000",
    "beto-cased-20000",
    "robertuito-cased-574k",
    "robertuito-cased",
    "roberta-bne",
]


df = pd.DataFrame(resume)
df.set_index("model", inplace=True)

df_mean = pd.DataFrame(mean_resume).set_index("model")
score_cols = ["context_hate mean_f1", "hate macro_f1", "sentiment macro_f1", "emotion macro_f1"]
score_without_chate_cols = ["hate macro_f1", "sentiment macro_f1", "emotion macro_f1"]

df.loc[order, "score"] = df_mean.loc[order, score_cols].mean(axis=1)
df.loc[order, "score2"] = df_mean.loc[order, score_without_chate_cols].mean(axis=1)
columns = [c for c in df.columns if "micro" not in c and "score" not in c] + ["score", "score2" ]

df.loc[order, columns]

'irony' not in  beto-cased-10000
'irony' not in  beto-cased-20000
'irony' not in  beto-cased-2500
'irony' not in  beto-cased-5000
'irony' not in  beto-uncased-10000
'irony' not in  beto-uncased-20000
'irony' not in  beto-uncased-2500
'irony' not in  beto-uncased-5000
'irony' not in  checkpoint-100k
'irony' not in  checkpoint-124k
'irony' not in  checkpoint-46k
'irony' not in  checkpoint-62k
'irony' not in  checkpoint-87k


,context_hate mean_f1,context_hate hate_f1,hate hateful_f1,hate macro_f1,sentiment macro_f1,emotion macro_f1,irony ironic_f1,irony macro_f1,score,score2
model,,,,,,,,,,
bertin,0.5559 +- 0.002,0.6713 +- 0.002,0.7444 +- 0.004,0.7692 +- 0.005,0.6659 +- 0.003,0.5217 +- 0.014,0.6089 +- 0.012,0.7131 +- 0.004,0.628175,0.652267
beto-uncased,0.5895 +- 0.006,0.6907 +- 0.005,0.7344 +- 0.007,0.7556 +- 0.011,0.6484 +- 0.005,0.5198 +- 0.006,0.5926 +- 0.011,0.7031 +- 0.007,0.628341,0.641274
beto-uncased-2500,0.5864 +- 0.004,0.6857 +- 0.006,0.7667 +- 0.008,0.7867 +- 0.011,0.6762 +- 0.006,0.5303 +- 0.008,NaN,NaN,0.644898,0.664394
beto-uncased-5000,0.5532 +- 0.012,0.6615 +- 0.008,0.7364 +- 0.004,0.7560 +- 0.009,0.6552 +- 0.005,0.5094 +- 0.017,NaN,NaN,0.618437,0.640197
beto-uncased-10000,0.5829 +- 0.004,0.6808 +- 0.005,0.7615 +- 0.007,0.7818 +- 0.006,0.6806 +- 0.005,0.5536 +- 0.010,NaN,NaN,0.649757,0.672043
beto-uncased-20000,0.5844 +- 0.007,0.6867 +- 0.005,0.7414 +- 0.015,0.7630 +- 0.011,0.6840 +- 0.006,0.5292 +- 0.015,NaN,NaN,0.640152,0.658732
robertuito-uncased-200k,0.5642 +- 0.005,0.6788 +- 0.006,0.7606 +- 0.007,0.7778 +- 0.012,0.6914 +- 0.003,0.5144 +- 0.007,0.6319 +- 0.019,0.7232 +- 0.006,0.636964,0.661216
beto-cased,0.5836 +- 0.005,0.6846 +- 0.006,0.7477 +- 0.008,0.7647 +- 0.010,0.6647 +- 0.003,0.5258 +- 0.008,0.6043 +- 0.012,0.7074 +- 0.007,0.634689,0.651723
beto-cased-2500,0.5768 +- 0.002,0.6817 +- 0.004,0.7477 +- 0.008,0.7675 +- 0.006,0.6769 +- 0.006,0.5238 +- 0.010,NaN,NaN,0.636231,0.656037


In [5]:
import pandas as pd
import numpy as np

resume = []
mean_resume = []
task_metrics = {
    "context_hate": ["eval_mean_f1", "eval_hate_f1"],
    "hate": ["eval_hateful_f1", "eval_macro_f1"],
    "sentiment": ["eval_macro_f1", "eval_micro_f1"],
    "emotion": ["eval_macro_f1", "eval_micro_f1"],
}

    
for model_name, output in outs.items():
    line = {
        "model": model_name, 
    }

    mean_line = {
        "model": model_name,
    }

    for task, metrics in task_metrics.items():
        try:
            for metric in metrics:
                arr = np.array([evaluation[metric] for evaluation in output[task]])
                metric_name = metric.replace("eval_", "")
                mean_line[task+" "+metric_name] = arr.mean()
                line[task+" "+metric_name] = f"{arr.mean():.4f} +- {arr.std():.3f}"
        except KeyError as e:
            print(e, "not in ", model_name)
            continue
    resume.append(line)
    mean_resume.append(mean_line)

order = [
    "bertin",
    "beto-cased",
    "beto-uncased",
    "beto-ft-1000",
    "beto-ft-2000",
    "beto-uncased-1000",
    "beto-uncased-2000",
    "beto-uncased-5000",
    "beto-uncased-5000-grito",
    "beto-uncased-4000-tpu",
    "beto-uncased-5000-tpu",
    "beto-uncased-15000",
    "beto-uncased-15000-last",
    "beto-cased-15000",
]


df = pd.DataFrame(resume)
df.set_index("model", inplace=True)

df_mean = pd.DataFrame(mean_resume).set_index("model")
score_cols = ["context_hate mean_f1", "hate hateful_f1", "sentiment macro_f1", "emotion macro_f1"]

df.loc[order, "score"] = df_mean.loc[order, score_cols].mean(axis=1)
columns = [c for c in df.columns if "micro" not in c]

df.loc[order, columns]

,context_hate mean_f1,context_hate hate_f1,hate hateful_f1,hate macro_f1,sentiment macro_f1,emotion macro_f1,score
model,,,,,,,
bertin,0.5443 +- 0.005,0.6613 +- 0.009,0.7270 +- 0.013,0.7539 +- 0.013,0.6650 +- 0.002,0.5245 +- 0.026,0.615193
beto-cased,0.5870 +- 0.007,0.6884 +- 0.005,0.7408 +- 0.013,0.7554 +- 0.021,0.6617 +- 0.005,0.5246 +- 0.016,0.628539
beto-uncased,0.5906 +- 0.004,0.6856 +- 0.006,0.7305 +- 0.006,0.7455 +- 0.014,0.6517 +- 0.002,0.5250 +- 0.014,0.624442
beto-ft-1000,0.5857 +- 0.003,0.6861 +- 0.002,0.7552 +- 0.011,0.7786 +- 0.012,0.6721 +- 0.001,0.5335 +- 0.012,0.636610
beto-ft-2000,0.5848 +- 0.005,0.6865 +- 0.003,0.7486 +- 0.003,0.7691 +- 0.010,0.6751 +- 0.005,0.5257 +- 0.010,0.633550
beto-uncased-1000,0.5950 +- 0.003,0.6934 +- 0.002,0.7451 +- 0.004,0.7703 +- 0.004,0.6717 +- 0.005,0.5364 +- 0.009,0.637067
beto-uncased-2000,0.5989 +- 0.004,0.6954 +- 0.002,0.7575 +- 0.009,0.7729 +- 0.015,0.6812 +- 0.001,0.5354 +- 0.005,0.643223
beto-uncased-5000,0.5788 +- 0.001,0.6775 +- 0.004,0.7623 +- 0.002,0.7854 +- 0.000,0.6748 +- 0.004,0.5399 +- 0.006,0.638945
beto-uncased-5000-grito,0.5848 +- 0.008,0.6866 +- 0.009,0.7615 +- 0.006,0.7738 +- 0.010,0.6865 +- 0.005,0.5522 +- 0.003,0.646247


- "last" es usando los mismos parámetros que en el paper de finetuning

No parecen ser los mejores!
